In [1]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import requests
from bs4 import BeautifulSoup

query = 'posts' # "site:https://www.ucvtscampuschronicle.com+posts&num=80"  # Modify the query as needed
url = f"https://www.google.com/search?q={query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers, timeout=1)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f"Failed to fetch URL: {url}")
    print(f"Status code: {response.status_code}")

    # load from results file
    with open('results.txt', 'r') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        print("Loaded from file")


Failed to fetch URL: https://www.google.com/search?q=posts
Status code: 429
Loaded from file


In [24]:
all_urls = [a['href'] for a in soup.find_all('a', href=True)]
campus_urls = [url for url in all_urls if 'https://www.ucvtscampuschronicle.com/single-post/' in url]
campus_urls

['https://www.ucvtscampuschronicle.com/single-post/practicing-self-love',
 'https://www.ucvtscampuschronicle.com/single-post/chatgpt-s-meteoric-rise-what-does-the-future-of-communication-look-like',
 'https://www.ucvtscampuschronicle.com/single-post/how-tiktok-is-quickening-the-fashion-trend-cycle',
 'https://www.ucvtscampuschronicle.com/single-post/how-shows-and-movies-are-filming-during-the-pandemic',
 'https://www.ucvtscampuschronicle.com/single-post/the-fashion-industry-s-not-so-secret-impact',
 'https://www.ucvtscampuschronicle.com/single-post/the-problem-with-spotify-wrapped',
 'https://www.ucvtscampuschronicle.com/single-post/misogyny-in-everyday-life',
 'https://www.ucvtscampuschronicle.com/single-post/open-letter-submission',
 'https://www.ucvtscampuschronicle.com/single-post/5-fruits-that-are-good-for-you',
 'https://www.ucvtscampuschronicle.com/single-post/stage-fright-a-monologue',
 'https://www.ucvtscampuschronicle.com/single-post/why-rising-gas-prices-are-important',
 'ht

In [25]:
authors = {
    'akash' : [
        "Akash Dubey",
        "AIT"
    ]
}

In [26]:
def get_author(short_name, full_name, school):
      author = f"""    {short_name}:
      name:           {full_name}
      site:           https://www.wowthemes.net
      avatar:         /assets/images/anonymous.jpg
      bio:            "Hi, I am {full_name}!"
      email:          no_reply@ucvts.org
      twitter:        https://twitter.com/
      school:         {school}
      grade:          Alumni"""
      return author

In [27]:
from datetime import datetime
import os
from dateutil import parser

def parse_datetime(date_string):
    try:
        parsed_date = parser.parse(date_string)
        return parsed_date.strftime("%Y-%m-%d")
    except ValueError as e:
        print(f"Failed to parse datetime: {str(e)}")
        try:
            return datetime.strptime(date, "%B %d, %Y").strftime("%Y-%m-%d")
        except ValueError:
            return datetime.strptime(date, "%b %d, %Y").strftime("%Y-%m-%d")
        return None

def save_image(url, name):
    # Send a request to the image URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the file name from the URL
        filename = name + '.' +  os.path.basename(url).split('.')[-1]

        # Specify the file path where the image will be saved
        filepath = os.path.join("assets/images", filename)

        print(f"Downloading file from {url}")
        print(f"Saving the file as {filename}")

        # Save the image to the specified file path
        with open(filepath, "wb") as file:
            file.write(response.content)
        print("Image downloaded and saved successfully.")
        return filepath
    
    print("Failed to download the image. Status code:", response.status_code)
    return None

def save_file(url):
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    else:
        print(f"Failed to fetch URL: {url}")
        return None

    article = soup.find(id='content-wrapper')
    image_url = article.find_all('img')[0].get('data-pin-media')
    title = article.find('h1').text
    print(title)
    text_list = [a.text for a in article.find_all('p')]

    text = '\n\n'.join(text_list)
    meta_data = [a.text for a in article.find_all('li')]
    try:
        full_name = meta_data[0].split(',')[0]
        school = meta_data[0].split(',')[1]
    except IndexError:
        print(meta_data)
        full_name = input("Enter full name: ")
        school = input("Enter School: ")
    
    short_name = full_name.split(' ')[0].lower()
    date = meta_data[3]
    category = meta_data[-1]

    # Convert title to valid filename
    short_title = title.lower().replace(" ", "-")

    good_file = "".join([char for char in short_title if char.isalnum() or char == '-'])

    path = save_image(image_url, good_file)

    article = f"""---
layout: post
title:  "{title}"
author: {short_name}
categories: [ {category} ]
image: {path}
---

{text}

"""

    authors[short_name] = [full_name, school, get_author(short_name, full_name, school)]

    # Parse the original date
    formatted_date = parse_datetime(date)


    # Create the file name by using the formatted date and title
    file_name = f"{formatted_date}-{good_file}.md"
    print(article, file=open(f"_posts/{file_name}", "w", encoding="ascii", errors="ignore"))
    return True
    

In [28]:
for result in reversed(campus_urls):
    try:
        save_file(result)
    except:
        print("Error with: " + result)
        continue
authors

Error with: https://www.ucvtscampuschronicle.com/single-post/what-lead-to-the-supreme-court-s-politicization
A Controversial Fight Between Two Controversial Youtubers: Logan Paul vs KSI II
Saving the file as a-controversial-fight-between-two-controversial-youtubers-logan-paul-vs-ksi-ii.jpg
Image downloaded and saved successfully.
English, A Frankenstein Monster of a Language
Saving the file as english-a-frankenstein-monster-of-a-language.png
Image downloaded and saved successfully.
The Staggering Decline in Marvel's Cinematic Universe
Saving the file as the-staggering-decline-in-marvels-cinematic-universe.webp
Image downloaded and saved successfully.
Eddie Van Halen Obituary
Saving the file as eddie-van-halen-obituary.jpg
Image downloaded and saved successfully.
Why You Should Watch the Marvel Movies
Saving the file as why-you-should-watch-the-marvel-movies.jpg
Image downloaded and saved successfully.
Interesting Facts About the Human Memory
Saving the file as interesting-facts-about-t

{'akash': ['Akash Dubey', 'AIT'],
 'alan': ['Alan Ostrovsky',
  ' AIT',
  '    alan:\n      name:           Alan Ostrovsky\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Alan Ostrovsky!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'joe': ['Joe Baldwin',
  ' UCTech',
  '    joe:\n      name:           Joe Baldwin\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Joe Baldwin!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'omolola': ['Omolola Falana',
  ' MHS',
  '    omolola:\n      name:           Omolola Falana\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n

In [ ]:
url = "https://www.ucvtscampuschronicle.com/single-post/2020/05/17/english-a-frankenstein-monster-of-a-language-1"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f"Failed")

In [ ]:
save_file("https://www.ucvtscampuschronicle.com/single-post/2020/05/17/english-a-frankenstein-monster-of-a-language-1")

Saving the file as english,-a-frankenstein-monster-of-a-language.png
Image downloaded and saved successfully.


True

In [ ]:
article = soup.find(id='content-wrapper')
article

<div id="content-wrapper"><div class="mbKyhH fsZ7OF mFmpGg blog-post-page-font" data-hook="post-page"><div class="KYa5Ob"><div></div><div><div><div><div class="EcwjNF"><article class="j3D9DG blog-post-page-font blog-card-background-color blog-card-border-color blog-text-color" data-hook="post" style="border-width:0"><div class="OSxNQL"><div class="OFA52E"><div class="IeNfNP"><ul class="xUuoH9 v0bFi9"><li class="F4tRtJ"><span class="" style="pointer-events:none"><div class="lBv2XN blog-text-color blog-link-hover-color avatar blog-post-text-color"><span class="tQ0Q1A user-name dlINDG" data-hook="user-name" title="Joe Baldwin, UCTech">Joe Baldwin, UCTech</span></div></span></li><li class="F4tRtJ"><div class="Ki_0_s rmqd9q blog-separator-background-color"></div></li><li class="F4tRtJ"><ul class="xUuoH9 MBUSKJ"><li class="UZa2Xr"><span class="post-metadata__date time-ago" data-hook="time-ago" title="May 17, 2020">May 17, 2020</span></li><li class="UZa2Xr"><div class="Ki_0_s rmqd9q blog-sepa

In [ ]:
article.text

'Joe Baldwin, UCTechMay 17, 20204 min readEnglish, A Frankenstein Monster of a Language English is a weird language. Any student who has had to memorize strange spelling anomalies or had to read aloud in class only to trip over a word with two different meanings and entirely different pronunciations can attest to this. For example, though they masquerade as the same word, lead (as in leader) and lead (as in the element) sound different when spoken. Taken to the extreme, sentences like “Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo” make complete grammatical sense. And, perhaps, consider that English is one of the only European languages in which nouns are not assigned gender.  There is a reason for these idiosyncrasies, though. The short answer is that English is a mutt of a language cobbled together from no less than four other languages. The long answer, however, requires a bit of history. First off, if you have ever been in a science class, then you are probably fa

In [ ]:
image_url = article.find_all('img')[0].get('data-pin-media')

'https://static.wixstatic.com/media/0089f9_e01e6a6db883427ea928ae3d6587f87d~mv2.png/v1/fill/w_960,h_540,al_c,q_90/0089f9_e01e6a6db883427ea928ae3d6587f87d~mv2.png'

In [ ]:
title = article.find('h1').text
title

'English, A Frankenstein Monster of a Language'

In [ ]:
text_list = [a.text for a in article.find_all('p')]

text = '\n'.join(text_list)
text

' \nEnglish is a weird language. Any student who has had to memorize strange spelling anomalies or had to read aloud in class only to trip over a word with two different meanings and entirely different pronunciations can attest to this. For example, though they masquerade as the same word, lead (as in leader) and lead (as in the element) sound different when spoken. Taken to the extreme, sentences like “Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo” make complete grammatical sense. And, perhaps, consider that English is one of the only European languages in which nouns are not assigned gender. \n \nThere is a reason for these idiosyncrasies, though. The short answer is that English is a mutt of a language cobbled together from no less than four other languages. The long answer, however, requires a bit of history.\n \nFirst off, if you have ever been in a science class, then you are probably familiar with words like photosynthesis or Yttrium. You may already know that 

In [ ]:
meta_data = [a.text for a in article.find_all('li')]
full_name = meta_data[0].split(',')[0]
school = meta_data[0].split(',')[1]
short_name = full_name.split(' ')[0].lower()
date = meta_data[3]
category = meta_data[-1]

In [ ]:
article = f"""---
layout: post
title:  "{title}"
author: {short_name}
categories: [ {category} ]
image: assets/images/3.jpg
---

{text}

"""
article

'---\nlayout: post\ntitle:  "English, A Frankenstein Monster of a Language"\nauthor: joe\ncategories: [ News ]\nimage: assets/images/3.jpg\n---\n\n \nEnglish is a weird language. Any student who has had to memorize strange spelling anomalies or had to read aloud in class only to trip over a word with two different meanings and entirely different pronunciations can attest to this. For example, though they masquerade as the same word, lead (as in leader) and lead (as in the element) sound different when spoken. Taken to the extreme, sentences like “Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo” make complete grammatical sense. And, perhaps, consider that English is one of the only European languages in which nouns are not assigned gender. \n \nThere is a reason for these idiosyncrasies, though. The short answer is that English is a mutt of a language cobbled together from no less than four other languages. The long answer, however, requires a bit of history.\n \nFirst o

In [32]:
for key, value in authors.items():
    print(value[-1], file=open("authors.txt", "a"))

In [1]:
authors = {'akash': ['Akash Dubey', 'AIT'],
 'alan': ['Alan Ostrovsky',
  ' AIT',
  '    alan:\n      name:           Alan Ostrovsky\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Alan Ostrovsky!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'joe': ['Joe Baldwin',
  ' UCTech',
  '    joe:\n      name:           Joe Baldwin\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Joe Baldwin!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'omolola': ['Omolola Falana',
  ' MHS',
  '    omolola:\n      name:           Omolola Falana\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Omolola Falana!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'haralambos': ['Haralambos Liberatos',
  ' UCTech',
  '    haralambos:\n      name:           Haralambos Liberatos\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Haralambos Liberatos!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'victoria': ['Victoria Gee',
  ' MHS',
  '    victoria:\n      name:           Victoria Gee\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Victoria Gee!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'laura': ['Laura Felder',
  ' UCTech',
  '    laura:\n      name:           Laura Felder\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Laura Felder!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'rebecca': ['Rebecca Lee',
  'AIT',
  '    rebecca:\n      name:           Rebecca Lee\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Rebecca Lee!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:         AIT\n      grade:          Alumni'],
 '': [' Sara Schwartz',
  ' AAHS',
  '    :\n      name:            Sara Schwartz\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am  Sara Schwartz!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'audrey': ['Audrey Miller',
  ' UCTech',
  '    audrey:\n      name:           Audrey Miller\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Audrey Miller!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'natalie': ['Natalie Lee',
  ' MHS',
  '    natalie:\n      name:           Natalie Lee\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Natalie Lee!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'dylan': ['Dylan Intwala',
  ' AIT',
  '    dylan:\n      name:           Dylan Intwala\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Dylan Intwala!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'alondra': ['Alondra Martinez',
  ' UCTech',
  '    alondra:\n      name:           Alondra Martinez\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Alondra Martinez!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'ernest': ['Ernest Romero',
  ' APA',
  '    ernest:\n      name:           Ernest Romero\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Ernest Romero!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          APA\n      grade:          Alumni'],
 'gregory': ['Gregory Martinez',
  ' AAHS',
  '    gregory:\n      name:           Gregory Martinez\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Gregory Martinez!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'maya': ['Maya Dias',
  ' AIT',
  '    maya:\n      name:           Maya Dias\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Maya Dias!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'shreya': ['Shreya Nara',
  ' AIT',
  '    shreya:\n      name:           Shreya Nara\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Shreya Nara!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'ella': ['Ella Platts',
  ' UCTech',
  '    ella:\n      name:           Ella Platts\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Ella Platts!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'monica': ['Monica Manescu',
  ' AAHS',
  '    monica:\n      name:           Monica Manescu\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Monica Manescu!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'diya': ['Diya Nainwal',
  ' AAHS',
  '    diya:\n      name:           Diya Nainwal\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Diya Nainwal!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'kenna': ['Kenna Srivastava',
  ' MHS',
  '    kenna:\n      name:           Kenna Srivastava\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Kenna Srivastava!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'saniya': ['Saniya Harlalka',
  ' AAHS',
  '    saniya:\n      name:           Saniya Harlalka\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Saniya Harlalka!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'justin': ['Justin Zimmerman',
  ' AIT',
  '    justin:\n      name:           Justin Zimmerman\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Justin Zimmerman!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'sarah': ['Sarah Bauman',
  ' APA',
  '    sarah:\n      name:           Sarah Bauman\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Sarah Bauman!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          APA\n      grade:          Alumni'],
 'kaitlyn': ['Kaitlyn Landaverde',
  ' APA',
  '    kaitlyn:\n      name:           Kaitlyn Landaverde\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Kaitlyn Landaverde!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          APA\n      grade:          Alumni'],
 'mattia': ['Mattia Leingang',
  ' AIT',
  '    mattia:\n      name:           Mattia Leingang\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Mattia Leingang!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'eric': ['Eric Yu',
  ' AIT',
  '    eric:\n      name:           Eric Yu\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Eric Yu!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'megan': ['Megan Lahetta',
  ' UCTech',
  '    megan:\n      name:           Megan Lahetta\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Megan Lahetta!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'daivya': ['Daivya Sadhvani',
  ' AIT',
  '    daivya:\n      name:           Daivya Sadhvani\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Daivya Sadhvani!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'jaime': ['Jaime Paladi',
  ' AIT',
  '    jaime:\n      name:           Jaime Paladi\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Jaime Paladi!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'akwele': ['Akwele Lokko',
  ' AAHS',
  '    akwele:\n      name:           Akwele Lokko\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Akwele Lokko!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'patrick': ['Patrick Mochnal',
  ' AIT',
  '    patrick:\n      name:           Patrick Mochnal\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Patrick Mochnal!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'haley': ['Haley Richardson',
  ' UCTech',
  '    haley:\n      name:           Haley Richardson\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Haley Richardson!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni'],
 'anusha': ['Anusha Iyer',
  ' AIT',
  '    anusha:\n      name:           Anusha Iyer\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Anusha Iyer!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'chloe': ['Chloe Gavula',
  ' APA',
  '    chloe:\n      name:           Chloe Gavula\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Chloe Gavula!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          APA\n      grade:          Alumni'],
 'sara': ['Sara Schwartz',
  ' AAHS',
  '    sara:\n      name:           Sara Schwartz\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Sara Schwartz!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'ahmad': ['Ahmad Khizar',
  ' AAHS',
  '    ahmad:\n      name:           Ahmad Khizar\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Ahmad Khizar!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AAHS\n      grade:          Alumni'],
 'ana': ['Ana Obradovic',
  ' AIT',
  '    ana:\n      name:           Ana Obradovic\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Ana Obradovic!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'rachel': ['Rachel Bataille',
  ' MHS',
  '    rachel:\n      name:           Rachel Bataille\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Rachel Bataille!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'lilly': ['Lilly Resnick',
  'AIT',
  '    lilly:\n      name:           Lilly Resnick\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Lilly Resnick!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:         AIT\n      grade:          Alumni'],
 'ivy': ['Ivy Liu',
  ' MHS',
  '    ivy:\n      name:           Ivy Liu\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Ivy Liu!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          MHS\n      grade:          Alumni'],
 'anonymous': ['Anonymous',
  'Unknown',
  '    anonymous:\n      name:           Anonymous\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Anonymous!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:         Unknown\n      grade:          Alumni'],
 'sophia': ['Sophia Mekbeb',
  ' APA',
  '    sophia:\n      name:           Sophia Mekbeb\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Sophia Mekbeb!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          APA\n      grade:          Alumni'],
 'mila': ['Mila Talati',
  ' AIT',
  '    mila:\n      name:           Mila Talati\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Mila Talati!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          AIT\n      grade:          Alumni'],
 'gissele': ['Gissele Hernandez',
  ' UCTech',
  '    gissele:\n      name:           Gissele Hernandez\n      site:           https://www.wowthemes.net\n      avatar:         /assets/images/anonymous.jpg\n      bio:            "Hi, I am Gissele Hernandez!"\n      email:          no_reply@ucvts.org\n      twitter:        https://twitter.com/\n      school:          UCTech\n      grade:          Alumni']}

In [36]:
# Generate individual author pages from authors dictionary
for short, value in authors.items():
    # Read template file
    with open('_pages/author-sal.html', 'r') as f:
        template = f.read()

    template = template.replace('akash-dubey', value[0].lower().replace(" ", "-"))
    
    # Replace akash with short
    template = template.replace('akash', short)

    # Replace Akash with value[0]
    template = template.replace('Akash', value[0])

    # save new file
    with open('_pages/author-' + short + '.html', 'w') as f:
        f.write(template)